In [1]:

import pickle
import time
import datetime
import importlib
import normalizerFunctions 
normalizerFunctions = importlib.reload(normalizerFunctions)
from normalizerFunctions import Token_Classifier
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils import data
from Normalization_Dataset import Normalization
from sklearn.model_selection import train_test_split
from transformers import get_linear_schedule_with_warmup


In [2]:
with open('pickled_archimob.pkl', 'rb') as file:
    archimob_corpus = pickle.load(file)

In [3]:
# check devixe
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(torch.backends.mps.is_available())

True


In [4]:
# define sequence padding 
def pad(batch):
    '''Pads to the longest sample'''
    f = lambda x: [sample[x] for sample in batch]
    words = f(0)
    is_heads = f(2)
    labels = f(3)
    seqlens = f(-1)
    maxlen = np.array(seqlens).max()
    f = lambda x, seqlen: [sample[x] + [0] * (seqlen - len(sample[x])) for sample in batch] # 0: <pad>
    x = f(1, maxlen)
    y = f(-2, maxlen)
    f = torch.LongTensor
    return words, f(x), is_heads, labels, f(y), seqlens

In [5]:
def get_Dataloader(corpus):
    labels = ["<pad>"] + list(corpus.labels)
    label2idx = {label:idx for idx, label in enumerate(labels)}
    idx2label = {idx:label for idx, label in enumerate(labels)}
    # create training / validation split and load data into batches 
    train_data, val_data = train_test_split(corpus.word_norm_pairs)
    train_dataset = Normalization(train_data, label2idx, corpus.multigrams)
    val_dataset = Normalization(val_data,label2idx, corpus.multigrams)

    train_iter = data.DataLoader(dataset=train_dataset,
                                batch_size=8,
                                shuffle=True,
                                num_workers=0,
                                collate_fn=pad)
    val_iter = data.DataLoader(dataset=val_dataset,
                                batch_size=8,
                                shuffle=False,
                                num_workers=0,
                                collate_fn=pad)
    print("Data loaded")
    return train_iter, val_iter, label2idx, idx2label

In [6]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [7]:
def get_normalizer(corpus,name):
    train_iter, val_iter, label2idx,idx2label = get_Dataloader(corpus)
    model = Token_Classifier(label2idx,idx2label)
    model.to(device)
    if torch.cuda.device_count() > 1:
        print(True)
        model = nn.DataParallel(model)
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    optimizer = optim.Adam(model.parameters(), lr = 0.0001)
    epochs = 4
    # performance and monitoring metrics 
    training_stats = []
    total_t0 = time.time()
    total_steps = len(train_iter) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, # Default value in run_glue.py
                                                num_training_steps = total_steps)
    # Full training loop
    for epoch in range(0,epochs):
        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch + 1, epochs))
        print('Training...')
        t0 = time.time()
        total_train_loss = 0
        model.train()
        for step, batch in enumerate(train_iter):
            # Progress update every 500 batches.
            if step % 500 == 0 and not step == 0:
                elapsed = format_time(time.time() - t0)
                print('Batch {} of {}.Elapsed: {:}.'.format(step, len(train_iter), elapsed))
            words, x, is_heads, labels, y, seqlens = batch
            _y = y # for monitoring
            optimizer.zero_grad()
            logits, y, _ = model(x, y) # logits: (N, L, VOCAB), y: (N, L)
            logits = logits.view(-1, logits.shape[-1]) # (N*L, VOCAB)
            y = y.view(-1)  # (N*L,)
            loss = criterion(logits, y)
            total_train_loss += loss.item()
            loss.backward()
            optimizer.step()
            scheduler.step()
            if step%1000==0: # monitoring
                print("step: {}, loss: {}".format(step, loss.item()))
        avg_train_loss = total_train_loss / len(train_iter)
        training_time = format_time(time.time() - t0)
        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epoch took: {:}".format(training_time))
        # Validation: 
        print("")
        print("Running Validation...")
        t0 = time.time()
        model.eval()
        # total_eval_loss = 0
        total = 0
        hits = 0
        words_unnormed = 0
        with open("norm_test_"+name+".txt", 'w') as fout: # generates a results file with the word, the true label, and the prediction
            for batch in val_iter:
                with torch.no_grad():        
                    b_utterances, x, b_is_heads, b_labels, y, seqlens = batch
                    _, _, b_predictions = model(x, y) # logits: (N, L, VOCAB), y: (N, L)
                    b_predictions = b_predictions.detach().cpu().numpy() # pred_ids.cpu().numpy().tolist() alternative?
                    assert len(b_utterances)==len(b_labels)== len(b_predictions)                
                    for utterance, utterance_labels, utterance_preds, is_heads in zip(b_utterances, b_labels, b_predictions, b_is_heads):
                        utterance_preds = [pred for head, pred in zip(is_heads, utterance_preds) if head == 1]
                        for pred in utterance_preds:
                            try:
                                test = idx2label[pred]
                            except KeyError:
                                idx2label[pred] = '<pad>'
                        preds = [idx2label[pred] for pred in utterance_preds]
                        words = utterance.split()
                        labels = utterance_labels.split()
                        assert len(preds)==len(words)== len(labels)
                        for w, l, p in zip(words[1:-1], labels[1:-1], preds[1:-1]):
                            if w == l:
                                words_unnormed += 1
                            if l == p:
                                hits += 1
                            total += 1
        accuracy = 100*hits/total
        unnormed = 100*words_unnormed/total
        print("Epoch {} accuracy: ".format(epoch+1),accuracy)
        print(unnormed)
        Err_Red_rate = (accuracy - unnormed)/(100 - unnormed) # all are percentages
        print(Err_Red_rate)
        print("Epoch {} error reduction rate: ".format(epoch+1),Err_Red_rate)
        training_stats.append(
            {
                'epoch': epoch + 1,
                'Training Loss': avg_train_loss,
                # 'Valid. Loss': avg_val_loss,
                #'Validation Time': validation_time,
                'Error Reduction': Err_Red_rate,
                'Training Time': training_time
            }
        )
        
    print("")
    print("Training complete!")
    print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
    torch.save(model, 'token_classifier_'+name+'.pt')


In [ ]:
# name must be changed on each run to avoid overwriting existing model
get_normalizer(archimob_corpus,'archimob_v2')